# Travel time

Load zipcode data

In [104]:
zipcode = pd.read_excel('../data/regionsopdelt-postnumme-pr.-22-06-2021.xlsx', header=1)
zipcode.head()

,AMTS_NR,ADRESSERINGSNAVN,KOMMUNE_NR,ADRESSERINGSNAVN_1,POSTNR,BYNAVN
0,1084,Region Hovedstaden,101,Københavns Kommune,783,Facility
1,1084,Region Hovedstaden,101,Københavns Kommune,784,H&M
2,1084,Region Hovedstaden,101,Københavns Kommune,785,Bestseller
3,1084,Region Hovedstaden,169,Høje-Taastrup Kommune,800,Høje Taastrup
4,1084,Region Hovedstaden,101,Københavns Kommune,900,København C


In [115]:
zipcode['ADRESSERINGSNAVN_1'] = zipcode['ADRESSERINGSNAVN_1'].str.replace(' Kommune', '')
for muncip in ['Københavns', 'Nordfyns', 'Vesthimmerlands']:
    zipcode.loc[zipcode['ADRESSERINGSNAVN_1'] == muncip, 'ADRESSERINGSNAVN_1'] = zipcode.loc[zipcode['ADRESSERINGSNAVN_1'] == muncip, 'ADRESSERINGSNAVN_1'].replace(muncip, muncip[:-1])

In [185]:
muncip = zipcode.groupby('ADRESSERINGSNAVN_1').sample(1)
muncip = muncip.loc[~(muncip['ADRESSERINGSNAVN_1'] == 'Christiansø')]
muncip

,AMTS_NR,ADRESSERINGSNAVN,KOMMUNE_NR,ADRESSERINGSNAVN_1,POSTNR,BYNAVN
976,1083,Region Syddanmark,580,Aabenraa,6200,Aabenraa
1335,1081,Region Nordjylland,851,Aalborg,9230,Svenstrup J
1194,1082,Region Midtjylland,751,Aarhus,8270,Højbjerg
531,1084,Region Hovedstaden,165,Albertslund,2600,Glostrup
633,1084,Region Hovedstaden,201,Allerød,3400,Hillerød
...,...,...,...,...,...,...
1085,1083,Region Syddanmark,630,Vejle,7120,Vejle Øst
1405,1081,Region Nordjylland,820,Vesthimmerland,9640,Farsø
1297,1082,Region Midtjylland,791,Viborg,8831,Løgstrup
832,1085,Region Sjælland,390,Vordingborg,4780,Stege


In [186]:
import pgeocode
import requests
import json

geo = pgeocode.Nominatim('dk')
geo.query_postal_code("8200")

postal_code                     8200
country_code                      DK
place_name                  Aarhus N
state_name        Region Midtjylland
state_code                        82
county_name           Aarhus Kommune
county_code                      751
community_name                   NaN
community_code                   NaN
latitude                     56.1663
longitude                    10.1987
accuracy                         1.0
Name: 0, dtype: object

In [214]:
df = pd.DataFrame(columns=['orig', 'dest', 'distance', 'duration'])

i = 0
for _, zip1 in muncip.iterrows():
    for _, zip2 in muncip.iterrows():
        if (zip1['POSTNR'] < 1200) or (zip2['POSTNR'] < 1000):
            continue
        zip1_lat_lon = geo.query_postal_code(zip1['POSTNR']).loc[['latitude', 'longitude']]
        zip2_lat_lon = geo.query_postal_code(zip2['POSTNR']).loc[['latitude', 'longitude']]
        
        if zip1_lat_lon.isna().any() or zip2_lat_lon.isna().any():
            continue
        
        r = requests.get(f"http://router.project-osrm.org/route/v1/driving/{zip1_lat_lon['longitude']},{zip1_lat_lon['latitude']};{zip2_lat_lon['longitude']},{zip2_lat_lon['latitude']}?overview=false")
        
        routes = json.loads(r.content)
        print(i, zip1['ADRESSERINGSNAVN_1'], zip2['ADRESSERINGSNAVN_1'], routes.get('routes')[0].get('distance'), routes.get('routes')[0].get('duration'))
        df = df.append({'orig': zip1['ADRESSERINGSNAVN_1'],
                   'dest': zip2['ADRESSERINGSNAVN_1'],
                   'distance': routes.get('routes')[0].get('distance'),
                   'duration': routes.get('routes')[0].get('duration')},
                 ignore_index=True)
        i += 1
        #df.append({'orig': zip1['POSTNR'], 'dest': zip2['POSTNR'], 

0 Aabenraa Aabenraa 0 0
1 Aabenraa Aalborg 257499.7 9894.1
2 Aabenraa Aarhus 154397 6162.2
3 Aabenraa Albertslund 278055.8 10909.7
4 Aabenraa Allerød 313118.5 12584.5


KeyboardInterrupt: 

In [213]:
df.to_csv('zipcodes.csv')